## Importação das bibliotecas

In [1]:
# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# Plots e avaliação
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import umap.umap_ as umap
import os
import pandas as pd
from PIL import Image

# Ignora todos os avisos
import warnings
warnings.filterwarnings("ignore")

## Execute abaixo, apenas se não tiver o _dataset_

In [2]:
# import kagglehub

# # Download da versão mais recente

# path = kagglehub.dataset_download("aibloy/fairface")

# print("Path to dataset files:", path)

## Dicionário de Configuração

Contém todos os itens necessários para a execução do _pipeline_ completo. Desde tamanho de _batch_ e taxa de aprendizado até o tipo de modelo usado e localização do _dataset_.

In [ ]:
CONFIG = {
    'batch_size': 128,          
    'lr': 0.0001,                
    'epochs': 10,
    'num_classes': 7,            # Ex: FairFace (White, Black, Indian, East/SE Asian, Middle East, Hispanic)
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'data_dir': 'FairFace',     
    'model_name': 'vit_b_16',    # Opções: 'vit_b_16', 'resnet50', 'resnet34', 'resnet101', 'efficientnet', 'vgg16'
    'use_arcface': True,         # LIGA/DESLIGA o ArcFace
    'embedding_size': 512       
}

print(f'Executando {CONFIG["model_name"]} com ArcFace={CONFIG["use_arcface"]} em: {CONFIG["device"]}')

Executando resnet101 com ArcFace=True em: cuda


## Classe ArcFace

Usado como parte do modelo para o reconhecimento de faces.

In [5]:
class ArcFaceLayer(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50):
        super(ArcFaceLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label=None):
        # 1. Normaliza features e pesos 
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        
        if label is None:
            return cosine * self.s

        # 2. Aplica a margem angular apenas na classe correta
        # cos(theta + m) = cos(theta)cos(m) - sin(theta)sin(m)
        phi = cosine - self.m # Aproximação simplificada robusta para treinamento
        
        # One-hot encoding para aplicar a margem apenas no target
        one_hot = torch.zeros(cosine.size(), device=CONFIG['device'])
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        
        # Logits finais: classe correta ganha penalidade, forçando aprendizado
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        
        return output

## Classe ModelWrapper

Utilizada para extrair _embeddings_ a partir dos modelos. 

In [6]:
class ModelWrapper(nn.Module):
    def __init__(self, base_model, input_features, num_classes, use_arcface=False):
        super(ModelWrapper, self).__init__()
        self.features = base_model
        self.use_arcface = use_arcface
        
        # Camada de projeção para garantir tamanho fixo do embedding
        self.embedding_layer = nn.Linear(input_features, CONFIG['embedding_size'])
        self.bn = nn.BatchNorm1d(CONFIG['embedding_size']) # BN ajuda muito no ArcFace
        
        if use_arcface:
            self.classifier = ArcFaceLayer(CONFIG['embedding_size'], num_classes)
        else:
            self.classifier = nn.Linear(CONFIG['embedding_size'], num_classes)
        
    def forward(self, x, labels=None):
        # Extração de features
        x = self.features(x)
        x = x.view(x.size(0), -1) # Flatten
        
        # Gera Embedding
        embeddings = self.embedding_layer(x)
        embeddings = self.bn(embeddings)
        
        # Classificação (ArcFace precisa dos labels no treino)
        if self.use_arcface and labels is not None:
            logits = self.classifier(embeddings, labels)
        else:
            logits = self.classifier(embeddings)
            
        return logits, embeddings

## Instanciamento do Modelo

Dependendo da configuração selecionada na célula de CONFIG, é selecionada um modelo para treinamento. O modelo será baixado com os pesos padrões.

In [7]:
def get_model(model_name, num_classes):
    print(f"Carregando {model_name}...")
    
    if model_name == 'vit_b_16':
        # Vision Transformer SOTA
        base = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
        num_ftrs = base.heads.head.in_features
        base.heads = nn.Identity() # Remove a cabeça original

    elif model_name == 'resnet34':
        base = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        num_ftrs = base.fc.in_features
        base.fc = nn.Identity()
        
    elif model_name == 'resnet50':
        base = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        num_ftrs = base.fc.in_features
        base.fc = nn.Identity()
    
    elif model_name == 'resnet101':
        base = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
        num_ftrs = base.fc.in_features
        base.fc = nn.Identity()
        
    elif model_name == 'efficientnet':
        base = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        num_ftrs = base.classifier[1].in_features
        base.classifier = nn.Identity()
        
    elif model_name == 'vgg16':
        base = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        num_ftrs = base.classifier[6].in_features
        base.classifier[6] = nn.Identity()
        
    else:
        raise ValueError("Modelo desconhecido")

    model = ModelWrapper(base, num_ftrs, num_classes, CONFIG['use_arcface'])
    return model.to(CONFIG['device'])

## Treinamento

Nesta célua, é definida a função de treinamento do modelo, obtendo métricas interessantes ao longo das épocas e realizando todo o processo de ajuste de pesos. Ao final, retorna o modelo com os pesos treinados.

In [8]:
def train_model(model, dataloaders, criterion, optimizer):
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(CONFIG['epochs']):
        print(f'Epoch {epoch+1}/{CONFIG["epochs"]}')
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(CONFIG['device'])
                labels = labels.to(CONFIG['device'])

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    # ArcFace precisa dos labels no forward pass durante o treino
                    if phase == 'train' and CONFIG['use_arcface']:
                        logits, _ = model(inputs, labels)
                    else:
                        logits, _ = model(inputs) # Validação ou Softmax normal
                        
                    loss = criterion(logits, labels)
                    _, preds = torch.max(logits, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            if phase == 'train':
                history['train_loss'].append(epoch_loss)
            else:
                history['val_loss'].append(epoch_loss)
                history['val_acc'].append(epoch_acc.item())
                
    return model, history

## Gráficos de evolução

A partir das métricas de histórico obtidas no treinamento do modelo, esta célula plota os gráficos de evolução da _loss_ e acurácia do conjunto de treino e validação ao longo das épocas.

In [9]:
def plot_training_history(history):
    
    epochs = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize=(12, 5))

    # Subplot 1 - Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], 'b-', label='Treino Loss')
    plt.plot(epochs, history['val_loss'], 'r-', label='Validação Loss')
    plt.title('Evolução da Loss')
    plt.xlabel('Épocas')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Subplot 2 - Acurácia
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['val_acc'], 'g-', label='Validação Acc')
    plt.title('Evolução da Acurácia de Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

## Criação do _dataset_

O dataset FairFace é composto por duas pastas de imagens chamadas "_train_" e "_val_" e dois arquivos CSV dedicados a cada uma das pastas contendo informações sobre gênero, idade e raça das imagens presentes na base de dados. Nesta célula, extraímos apenas a informação relevante para o projeto, a raça. E deixamos a separação de treinamento e validação inalteradas em relação à forma que vieram no _dataset_.

In [10]:
class FairFaceDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, class_map=None):

        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
        # Cria mapeamento automático de classes (String -> Int) se não for fornecido
        if class_map is None:
            self.classes = sorted(self.annotations['race'].unique())
            self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        else:
            self.class_to_idx = class_map
            self.classes = list(class_map.keys())

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Obtém o nome da imagem do CSV.
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx]['file'])
        
        try:
            image = Image.open(img_name).convert('RGB')
        except (IOError, FileNotFoundError):
            # Fallback de segurança para não quebrar o treino por 1 imagem corrompida
            print(f'Aviso: Imagem não encontrada ou corrompida: {img_name}')
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        # Obtém o label e converte para índice
        race_label_str = self.annotations.iloc[idx]['race']
        label = self.class_to_idx[race_label_str]
        label = torch.tensor(label, dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label

def get_dataloaders():
    # Define os caminhos exatos
    BASE_DIR = './FairFace' # Ajuste conforme o ambiente
    TRAIN_CSV = os.path.join(BASE_DIR, 'train_labels.csv')
    VAL_CSV = os.path.join(BASE_DIR, 'val_labels.csv')     
    
    transforms_list = {
        'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2), 
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),
        'val': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),
    }

    print('Carregando datasets via CSV...')
    
    # 1. Cria Dataset de Treino
    # root_dir aponta para BASE_DIR pois o CSV do FairFace geralmente inclui o prefixo "train/" no nome do arquivo
    train_dataset = FairFaceDataset(csv_file=TRAIN_CSV, 
                                    root_dir=BASE_DIR, 
                                    transform=transforms_list['train'])
    
    # 2. Cria Dataset de Validação
    # Passa o class_map do treino para garantir que "Asian" seja o ID 0 em ambos
    val_dataset = FairFaceDataset(csv_file=VAL_CSV, 
                                  root_dir=BASE_DIR, 
                                  transform=transforms_list['val'],
                                  class_map=train_dataset.class_to_idx)

    dataloaders = {
        'train': DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=0),
        'val': DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=0)
    }
    
    class_names = train_dataset.classes
    print(f'Classes encontradas: {class_names}')
    print(f'Tamanho Treino: {len(train_dataset)} | Tamanho Val: {len(val_dataset)}')
    
    return dataloaders, class_names

## Avaliação

Esta célula é responsável por mostrar informações como acurácia, precisão e recall de classes específicas ou do conjunto como um todo. O UMAP também é gerado nesta célula para avaliação do balanceamento da base de dados prevista pelo modelo e a real.

In [11]:
def evaluate_deia(model, dataloader, class_names):
    model.eval()
    all_preds = []
    all_labels = []
    all_embeddings = []

    print('\nExtraindo Embeddings para Análise de Viés...')
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(CONFIG['device'])
            labels = labels.to(CONFIG['device'])
            logits, embeddings = model(inputs)
            _, preds = torch.max(logits, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_embeddings.extend(embeddings.cpu().numpy())

    # Métricas
    print(classification_report(all_labels, all_preds, target_names=class_names))
    
    # Visualização
    X = np.array(all_embeddings)
    y_true = np.array(all_labels)
    y_pred = np.array(all_preds)
    
    print('Gerando UMAP (pode demorar um pouco)...')
    reducer = umap.UMAP(n_neighbors=20, min_dist=0.1, metric='cosine')
    embedding_2d = reducer.fit_transform(X)
    
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))
    
    # Plot 1: Realidade (Labels Verdadeiros)
    scatter1 = ax[0].scatter(embedding_2d[:, 0], embedding_2d[:, 1], c=y_true, cmap='tab10', s=15, alpha=0.7)
    ax[0].set_title('Espaço Latente: Classes Reais (Ground Truth)')
    ax[0].legend(*scatter1.legend_elements(), title='Etnias')
    
    # Plot 2: Percepção do Modelo (Predições)
    scatter2 = ax[1].scatter(embedding_2d[:, 0], embedding_2d[:, 1], c=y_pred, cmap='tab10', s=15, alpha=0.7)
    ax[1].set_title('Espaço Latente: Predição do Modelo')
    
    # Destaque de Erros DEIA
    # Pontos onde Pred != Real podem ser circulados ou marcados
    
    plt.tight_layout()
    plt.show()

## _Pipeline_ de execução

Após a compartimentalização do código, a classe _main_ é responsável por executar todos os processos. Desde o carregamento do modelo, criação do _dataset_ e obtenção das métricas de avaliação do modelo. Altere o dicionário CONFIG para procurar resultados diferentes.

In [ ]:
if __name__ == "__main__":
    # Gera os conjuntos de treino e validação
    dataloaders, class_names = get_dataloaders()
    CONFIG['num_classes'] = len(class_names)
    
    # Carrega o modelo e define a função de perda e otimizador
    model = get_model(CONFIG['model_name'], CONFIG['num_classes'])
    # model.load_state_dict(torch.load('Checkpoints/vit_b_16_2.pth', map_location=CONFIG['device']))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=1e-4) # AdamW é melhor para ViT
    
    # Treinamento
    model, history = train_model(model, dataloaders, criterion, optimizer)

    # Salva o modelo treinado
    torch.save(model.state_dict(), f'Checkpoints/{CONFIG["model_name"]}.pth')

    # Plot dos resultados
    plot_training_history(history)
    # Plot do UMAP do treino
    evaluate_deia(model, dataloaders['train'], class_names)
    # Plot do UMAP da validação
    evaluate_deia(model, dataloaders['val'], class_names)


Carregando datasets via CSV...
Classes encontradas: ['Black', 'East Asian', 'Indian', 'Latino_Hispanic', 'Middle Eastern', 'Southeast Asian', 'White']
Tamanho Treino: 86744 | Tamanho Val: 10954
Carregando vgg16...
Epoch 1/10


KeyboardInterrupt: 